In [18]:
!ls ../../evaluations/hate_speech/task_b/

beto-combi.json			  robertuito-combi.json
beto-hierarchical-gamma-0.1.json  robertuito-hierarchical-gamma-0.1.json
beto-hierarchical-gamma-0.json	  robertuito-hierarchical-gamma-0.json
beto.json			  robertuito.json


In [41]:
import json
import pandas as pd

evaluations = {

}

for model_name, path in [
    ('beto', '../../evaluations/hate_speech/task_b/beto.json'),
    ('beto-hier', '../../evaluations/hate_speech/task_b/beto-hierarchical-gamma-0.1.json'),
    #('beto-hier-zero', '../../evaluations/hate_speech/task_b/beto-hierarchical-gamma-0.json'),
    ('beto-combi', '../../evaluations/hate_speech/task_b/beto-combi.json'),
    ('robertuito', '../../evaluations/hate_speech/task_b/robertuito.json'),
    ('robertuito-hier', '../../evaluations/hate_speech/task_b/robertuito-hierarchical-gamma-0.1.json'),
    #('robertuito-hier-zero', '../../evaluations/hate_speech/task_b/robertuito-hierarchical-gamma-0.json'),
    ('robertuito-combi', '../../evaluations/hate_speech/task_b/robertuito-combi.json'),
    ]:
    with open(path) as f:
        evaluations[model_name] = json.load(f)


for key, evals in evaluations.items():
    print(f"We have {len(evals['evaluations']['hate_speech'])} {key} evaluations")



dfs = []
for model_name, model_results in evaluations.items():
    model_evaluations = model_results["evaluations"]["hate_speech"]
    
    if not model_evaluations:
        continue
    
    df = pd.DataFrame(model_evaluations)

    df.columns = [x.split("_", 1)[1] if "_" in x else x for x in df.columns]
    
    mean_df = pd.DataFrame({
        f"{model_name} mean": df.mean(), 
        f"{model_name} std": df.std()
    })
    dfs.append(mean_df)

result_df = pd.concat(dfs, axis=1)
index = [
    'hs_f1', 'hs_precision', 'hs_recall', 'tr_f1',
    'ag_f1', 'macro_hs_f1_score',
    'emr_no_gating', 'emr', 
    'macro_f1',
]
result_df.loc[index, [c for c in result_df.columns if "std" not in c]]

We have 10 beto evaluations
We have 10 beto-hier evaluations
We have 10 beto-combi evaluations
We have 10 robertuito evaluations
We have 10 robertuito-hier evaluations
We have 10 robertuito-combi evaluations


,beto mean,beto-hier mean,beto-combi mean,robertuito mean,robertuito-hier mean,robertuito-combi mean
hs_f1,0.741171,0.734925,0.742136,0.765693,0.759455,0.770661
hs_precision,0.712582,0.743658,0.725866,0.761301,0.749329,0.768850
hs_recall,0.777879,0.727576,0.761515,0.772424,0.773939,0.774697
tr_f1,0.765226,0.758053,0.763441,0.792220,0.794922,0.808406
ag_f1,0.687683,0.674417,0.667843,0.697813,0.680564,0.696601
macro_hs_f1_score,0.771392,0.776030,0.776321,0.799745,0.792895,0.804141
emr_no_gating,0.684063,0.673875,0.697625,0.709688,0.692187,0.723125
emr,0.684812,0.703313,0.697625,0.709937,0.705938,0.723125
macro_f1,0.731360,0.722465,0.724473,0.751909,0.744980,0.758556


In [42]:
import scipy.stats

combi_emr = [evaluation["eval_emr"] for evaluation in evaluations["beto-combi"]["evaluations"]["hate_speech"]]
hier_emr = [evaluation["eval_emr"] for evaluation in evaluations["beto-hier"]["evaluations"]["hate_speech"]]
standard_emr = [evaluation["eval_emr"] for evaluation in evaluations["beto"]["evaluations"]["hate_speech"]]


Corremos un test de Kruskal para ver si la diferencia entre las medias es significativa


In [43]:
scipy.stats.kruskal(standard_emr, hier_emr, combi_emr)

KruskalResult(statistic=3.4920080142475607, pvalue=0.17446973214802494)

No lo es

In [44]:
import scipy.stats

combi_emr = [evaluation["eval_emr"] for evaluation in evaluations["robertuito-combi"]["evaluations"]["hate_speech"]]
hier_emr = [evaluation["eval_emr"] for evaluation in evaluations["robertuito-hier"]["evaluations"]["hate_speech"]]
standard_emr = [evaluation["eval_emr"] for evaluation in evaluations["robertuito"]["evaluations"]["hate_speech"]]

scipy.stats.kruskal(standard_emr, hier_emr, combi_emr)

KruskalResult(statistic=10.860138146167559, pvalue=0.004382793059181017)

En el caso de robertuito sí! wtf